In [241]:
import pandas as pd
import numpy as np

In [242]:
df_player = pd.read_csv('player_data_NBA.csv')
df_team = pd.read_csv('adv_team.csv')

/var/folders/qh/d4ryvm2x08z1j11n5h2qr6mr0000gn/T/ipykernel_19210/771092782.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_player = pd.read_csv('player_data_NBA.csv')


### Prepare NBA player data with attributes needed for the model

In [243]:
df_team['Team'] = df_team['Team'].str.replace('*', '')

In [244]:
list_drop_df_player = ['Eastern Conf All-Stars', 'Leb', 'Team Durant', 'Team Giannis', 'Team LeBron', 'Team Stephen', 'USA', 'Usa', 'Western Conf All-Stars', 'World']

In [245]:
df_player = df_player[~df_player['opponent_team_display_name'].isin(list_drop_df_player)]

In [246]:
df_player = df_player[~df_player['team_name'].isin(list_drop_df_player)]

In [247]:
df_player = df_player[['athlete_id', 'athlete_display_name', 'minutes', 'points', 
'field_goals_attempted', 'field_goals_made', 'three_point_field_goals_made', 
'free_throws_attempted', 'season', 'team_display_name', 'opponent_team_display_name']]

In [248]:
df_player = df_player[~df_player['season'].isin([2024])]

### Prepare team level features for model

In [249]:
df_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           300 non-null    float64
 1   Team         310 non-null    object 
 2   Age          310 non-null    float64
 3   W            300 non-null    float64
 4   L            300 non-null    float64
 5   PW           310 non-null    int64  
 6   PL           310 non-null    int64  
 7   MOV          310 non-null    float64
 8   SOS          310 non-null    float64
 9   SRS          310 non-null    float64
 10  ORtg         310 non-null    float64
 11  DRtg         310 non-null    float64
 12  NRtg         300 non-null    float64
 13  Pace         310 non-null    float64
 14  FTr          310 non-null    float64
 15  3PAr         310 non-null    float64
 16  TS%          310 non-null    float64
 17  Unnamed: 17  0 non-null      float64
 18  eFG%         310 non-null    float64
 19  TOV%    

In [250]:
remove = ['League Average']
df_team = df_team[~df_team['Team'].isin(remove)]

In [251]:
df_team['Team'].value_counts().sort_index()

Team
Atlanta Hawks             10
Boston Celtics            10
Brooklyn Nets             10
Charlotte Bobcats          1
Charlotte Hornets          9
Chicago Bulls             10
Cleveland Cavaliers       10
Dallas Mavericks          10
Denver Nuggets            10
Detroit Pistons           10
Golden State Warriors     10
Houston Rockets           10
Indiana Pacers            10
Los Angeles Clippers      10
Los Angeles Lakers        10
Memphis Grizzlies         10
Miami Heat                10
Milwaukee Bucks           10
Minnesota Timberwolves    10
New Orleans Pelicans      10
New York Knicks           10
Oklahoma City Thunder     10
Orlando Magic             10
Philadelphia 76ers        10
Phoenix Suns              10
Portland Trail Blazers    10
Sacramento Kings          10
San Antonio Spurs         10
Toronto Raptors           10
Utah Jazz                 10
Washington Wizards        10
Name: count, dtype: int64

In [252]:
df_player['team_display_name'].value_counts().sort_index()

team_display_name
Atlanta Hawks             11107
Boston Celtics            11877
Brooklyn Nets             10806
Charlotte Bobcats          1102
Charlotte Hornets          9155
Chicago Bulls             10269
Cleveland Cavaliers       11065
Dallas Mavericks          11077
Denver Nuggets            11278
Detroit Pistons           10241
Golden State Warriors     11725
Houston Rockets           10981
Indiana Pacers            10828
LA Clippers               11266
Los Angeles Lakers        10837
Memphis Grizzlies         10912
Miami Heat                11618
Milwaukee Bucks           11524
Minnesota Timberwolves    10473
New Orleans Pelicans      10611
New York Knicks           10420
Oklahoma City Thunder     10970
Orlando Magic             10228
Philadelphia 76ers        10998
Phoenix Suns              10997
Portland Trail Blazers    11027
Sacramento Kings          10492
San Antonio Spurs         11157
Toronto Raptors           11420
Utah Jazz                 10991
Washington Wizards    

In [253]:
# Change Los Angeles Clippers to LA Clippers on df_team
df_team.loc[df_team['Team'] == 'Los Angeles Clippers', 'Team'] = 'LA Clippers'

In [254]:
df_team['Team_Year'] = df_team['Team'] + '_' + df_team['Year'].astype('str')

In [255]:
# for team of the target player
df_team_player = df_team[['Team_Year', 'Pace', 'DRB%', 'eFG%.1', 'ORB%']]

In [256]:
# for opponent of the target player
df_team_opp = df_team[['Team_Year', 'ORtg', 'eFG%', 'TOV%']]

In [257]:
df_player.head()

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,season,team_display_name,opponent_team_display_name
0,469,Rashard Lewis,9.0,3.0,2.0,1.0,1.0,0.0,2014,Miami Heat,San Antonio Spurs
1,1977,Chris Bosh,39.0,13.0,14.0,6.0,0.0,2.0,2014,Miami Heat,San Antonio Spurs
2,1966,LeBron James,41.0,31.0,21.0,10.0,3.0,9.0,2014,Miami Heat,San Antonio Spurs
3,9,Ray Allen,31.0,5.0,8.0,1.0,1.0,2.0,2014,Miami Heat,San Antonio Spurs
4,1987,Dwyane Wade,36.0,11.0,12.0,4.0,1.0,4.0,2014,Miami Heat,San Antonio Spurs


In [258]:
df_player['Player_Team_Year'] = df_player['team_display_name'] + '_' + df_player['season'].astype('str')

In [259]:
df_player.head()

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,season,team_display_name,opponent_team_display_name,Player_Team_Year
0,469,Rashard Lewis,9.0,3.0,2.0,1.0,1.0,0.0,2014,Miami Heat,San Antonio Spurs,Miami Heat_2014
1,1977,Chris Bosh,39.0,13.0,14.0,6.0,0.0,2.0,2014,Miami Heat,San Antonio Spurs,Miami Heat_2014
2,1966,LeBron James,41.0,31.0,21.0,10.0,3.0,9.0,2014,Miami Heat,San Antonio Spurs,Miami Heat_2014
3,9,Ray Allen,31.0,5.0,8.0,1.0,1.0,2.0,2014,Miami Heat,San Antonio Spurs,Miami Heat_2014
4,1987,Dwyane Wade,36.0,11.0,12.0,4.0,1.0,4.0,2014,Miami Heat,San Antonio Spurs,Miami Heat_2014


In [260]:
df_player['Player_Opp_Team_Year'] = df_player['opponent_team_display_name'] + '_' + df_player['season'].astype('str')

In [261]:
# drop extra columns
df_player.drop(['team_display_name', 'opponent_team_display_name', 'season'], axis=1, inplace=True)

In [262]:
df_player.head()

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Player_Team_Year,Player_Opp_Team_Year
0,469,Rashard Lewis,9.0,3.0,2.0,1.0,1.0,0.0,Miami Heat_2014,San Antonio Spurs_2014
1,1977,Chris Bosh,39.0,13.0,14.0,6.0,0.0,2.0,Miami Heat_2014,San Antonio Spurs_2014
2,1966,LeBron James,41.0,31.0,21.0,10.0,3.0,9.0,Miami Heat_2014,San Antonio Spurs_2014
3,9,Ray Allen,31.0,5.0,8.0,1.0,1.0,2.0,Miami Heat_2014,San Antonio Spurs_2014
4,1987,Dwyane Wade,36.0,11.0,12.0,4.0,1.0,4.0,Miami Heat_2014,San Antonio Spurs_2014


In [263]:
df_main = df_player.merge(df_team_player, left_on='Player_Team_Year', right_on='Team_Year', how='left', suffixes=('', '_player_team'))

In [264]:
df_main = df_main.merge(df_team_opp, left_on='Player_Opp_Team_Year', right_on='Team_Year', how='left', suffixes=('', '_player_opp'))

In [265]:
df_main.head()

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Player_Team_Year,Player_Opp_Team_Year,Team_Year,Pace,DRB%,eFG%.1,ORB%,Team_Year_player_opp,ORtg,eFG%,TOV%
0,469,Rashard Lewis,9.0,3.0,2.0,1.0,1.0,0.0,Miami Heat_2014,San Antonio Spurs_2014,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
1,1977,Chris Bosh,39.0,13.0,14.0,6.0,0.0,2.0,Miami Heat_2014,San Antonio Spurs_2014,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
2,1966,LeBron James,41.0,31.0,21.0,10.0,3.0,9.0,Miami Heat_2014,San Antonio Spurs_2014,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
3,9,Ray Allen,31.0,5.0,8.0,1.0,1.0,2.0,Miami Heat_2014,San Antonio Spurs_2014,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
4,1987,Dwyane Wade,36.0,11.0,12.0,4.0,1.0,4.0,Miami Heat_2014,San Antonio Spurs_2014,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5


In [266]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328236 entries, 0 to 328235
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   athlete_id                    328236 non-null  int64  
 1   athlete_display_name          328236 non-null  object 
 2   minutes                       271171 non-null  float64
 3   points                        271171 non-null  float64
 4   field_goals_attempted         271171 non-null  float64
 5   field_goals_made              271171 non-null  float64
 6   three_point_field_goals_made  271171 non-null  float64
 7   free_throws_attempted         271171 non-null  float64
 8   Player_Team_Year              328236 non-null  object 
 9   Player_Opp_Team_Year          328236 non-null  object 
 10  Team_Year                     328236 non-null  object 
 11  Pace                          328236 non-null  float64
 12  DRB%                          328236 non-nul

In [267]:
df_main[df_main['Team_Year'].isnull()]

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Player_Team_Year,Player_Opp_Team_Year,Team_Year,Pace,DRB%,eFG%.1,ORB%,Team_Year_player_opp,ORtg,eFG%,TOV%


In [268]:
df_main.describe()

,athlete_id,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Pace,DRB%,eFG%.1,ORB%,ORtg,eFG%,TOV%
count,3.282360e+05,271171.000000,271171.000000,271171.000000,271171.000000,271171.000000,271171.000000,328236.000000,328236.000000,328236.000000,328236.000000,328236.000000,328236.000000,328236.000000
mean,1.816822e+06,22.848623,10.144923,8.161489,3.747739,0.992459,2.159051,97.351751,76.508351,0.519390,23.511760,109.762230,0.520987,12.812100
std,1.763889e+06,10.725051,8.353191,5.791720,3.105024,1.369379,2.802464,2.994157,1.910528,0.021435,2.436853,4.264019,0.023884,0.971112
min,9.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.900000,71.000000,0.460000,17.900000,95.500000,0.456000,9.900000
25%,3.986000e+03,15.000000,4.000000,4.000000,1.000000,0.000000,0.000000,95.600000,75.300000,0.506000,21.600000,106.300000,0.504000,12.200000
50%,2.490589e+06,24.000000,8.000000,7.000000,3.000000,0.000000,2.000000,97.600000,76.500000,0.521000,23.400000,110.000000,0.522000,12.700000
75%,3.135046e+06,31.000000,15.000000,12.000000,6.000000,2.000000,3.000000,99.500000,77.800000,0.536000,25.000000,113.000000,0.538000,13.400000
max,5.075393e+06,65.000000,71.000000,50.000000,24.000000,14.000000,36.000000,105.100000,81.600000,0.576000,31.400000,119.400000,0.575000,16.000000


In [269]:
df_main.sample(5)

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Player_Team_Year,Player_Opp_Team_Year,Team_Year,Pace,DRB%,eFG%.1,ORB%,Team_Year_player_opp,ORtg,eFG%,TOV%
40126,862,Hedo Turkoglu,13.0,0.0,4.0,0.0,0.0,0.0,LA Clippers_2015,New York Knicks_2015,LA Clippers_2015,94.7,75.7,0.493,22.8,New York Knicks_2015,99.9,0.470,14.0
10720,996,Pau Gasol,32.0,29.0,15.0,10.0,0.0,14.0,Los Angeles Lakers_2014,New Orleans Pelicans_2014,Los Angeles Lakers_2014,98.7,71.0,0.513,20.2,New Orleans Pelicans_2014,107.2,0.495,12.9
114264,984,Tyson Chandler,20.0,5.0,4.0,1.0,0.0,4.0,Phoenix Suns_2017,Memphis Grizzlies_2017,Phoenix Suns_2017,100.3,76.4,0.525,26.0,Memphis Grizzlies_2017,107.7,0.491,12.1
74999,2596108,Michael Carter-Williams,NaN,NaN,NaN,NaN,NaN,NaN,Milwaukee Bucks_2016,Memphis Grizzlies_2016,Milwaukee Bucks_2016,94.2,73.1,0.510,24.9,Memphis Grizzlies_2016,105.4,0.477,12.3
249963,3064560,Luke Kornet,9.0,0.0,5.0,0.0,0.0,0.0,Chicago Bulls_2021,Indiana Pacers_2021,Chicago Bulls_2021,99.0,80.3,0.538,22.3,Indiana Pacers_2021,112.4,0.542,11.9


In [270]:
df_main = df_main.drop(columns=['Player_Team_Year', 'Player_Opp_Team_Year'])

In [271]:
df_main.head()

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Team_Year,Pace,DRB%,eFG%.1,ORB%,Team_Year_player_opp,ORtg,eFG%,TOV%
0,469,Rashard Lewis,9.0,3.0,2.0,1.0,1.0,0.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
1,1977,Chris Bosh,39.0,13.0,14.0,6.0,0.0,2.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
2,1966,LeBron James,41.0,31.0,21.0,10.0,3.0,9.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
3,9,Ray Allen,31.0,5.0,8.0,1.0,1.0,2.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5
4,1987,Dwyane Wade,36.0,11.0,12.0,4.0,1.0,4.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5


Calculate eFG% and TS%

eFG% = FG + (.5 * 3P) / FGA

TS% = PTS / (2*(FGA + .44 * FTA))

In [272]:
# Add eFG% for player
df_main['eFG%_player'] = (df_main['field_goals_made'] + (0.5 * df_main['three_point_field_goals_made'])) / df_main['field_goals_attempted']

In [273]:
# Add TS% for player
df_main['TS%_player'] = (df_main['points']) / (2 * (df_main['field_goals_attempted'] + (0.44 * df_main['free_throws_attempted'])))

In [274]:
df_main.head()

,athlete_id,athlete_display_name,minutes,points,field_goals_attempted,field_goals_made,three_point_field_goals_made,free_throws_attempted,Team_Year,Pace,DRB%,eFG%.1,ORB%,Team_Year_player_opp,ORtg,eFG%,TOV%,eFG%_player,TS%_player
0,469,Rashard Lewis,9.0,3.0,2.0,1.0,1.0,0.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5,0.750000,0.750000
1,1977,Chris Bosh,39.0,13.0,14.0,6.0,0.0,2.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5,0.428571,0.436828
2,1966,LeBron James,41.0,31.0,21.0,10.0,3.0,9.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5,0.547619,0.620994
3,9,Ray Allen,31.0,5.0,8.0,1.0,1.0,2.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5,0.187500,0.281532
4,1987,Dwyane Wade,36.0,11.0,12.0,4.0,1.0,4.0,Miami Heat_2014,91.2,73.0,0.511,20.6,San Antonio Spurs_2014,110.5,0.537,13.5,0.375000,0.399709


In [275]:
df_main.to_csv('NBA_points_model.csv')